### Sets Dictionary Views

A long time ago to iterate over the keys, values, or items of a dictionary. 
- .keys(), .values(), .items()  
These created and returned a list of these things

There were some issues
- the list is static
- list duplicates data - not good for large dictionaries, can be slow
- inefficient for membership testing (linear scan)

To help with iteration, some modifications were made:
- .iterkeys(), .itervalues(), iteritems()  
were introduced. They returned iterators

They didnt duplicate data and were more lightweight

However, this still did not help with membership testing

It also was not easy to answer questions such as, given d1 and d2, what keys are common to both? or what keys are in one but not the other?

That is because these are set questions

After all, keys have to be unique

So the **Key View** was introduced

Instead of keys() returning a list, and iterkeys() just being an iterator, what if keys() was a lightweight object that maintained a reference to the dictionary and implemented methods such as:

In [ ]:
__iter__ # iterable protocol
__contains__ # membership testing
__and__ # intersection of two views
__or__ # union of two views
__eq__ # same key in both views
__lt__ # is on set of keys a subset of the other
'etc

So now the key view not only behaves like an iterable, but also like a set

It does not "own" any data

#### Dictionary Views

Three ways we may want to view the data in a dictionary
- keys only
- values only
- key/value pairs 

#### Set Behavior

The keys() view always behaves like a (frozen) set
- since elements are unique (==) and hashable

The items() view may behave like a (frozen) set
- if the values are hashable
- uniqueness of tuples are guaranteed since keys  are unique

The values() view never behaves like a set
- values are not guaranteed to be unique or hashable

#### Modifying the dictionary while iterating over a view

be careful doing this
- modifying values usually not a problem
- modifying keys can lead to exceptions or worse disasters!

This is safe:

In [ ]:
for key in d.keys()
d[key] += 1

But this lead to an exception:

In [ ]:
for v in d.values():
    del d['a']

Python does not allow modifying the size of the underlying dictionary while iterating over a view

You technically can modify the keys as long as you do not change the size of the dictionary
- but dont do it!

This is what the Python documentation has to say about it:  
*Iterating views while adding or deleting entries in he dictionary may raise a RuntimeError or fail to iterate over all entries*

#### Code Examples

In [2]:
d = {'a': 1, 'b': 2}

In [3]:
keys = d.keys()
values = d.values()
items = d.items()

In [4]:
print(id(keys), id(values), id(items))

2771200829288 2771200830920 2771200830008


In [6]:
print(keys)
print(values)
print(items)

dict_keys(['a', 'b'])
dict_values([1, 2])
dict_items([('a', 1), ('b', 2)])


In [7]:
d['z'] = 10

In [8]:
print(keys)
print(values)
print(items)

dict_keys(['a', 'b', 'z'])
dict_values([1, 2, 10])
dict_items([('a', 1), ('b', 2), ('z', 10)])


In [9]:
print(id(keys), id(values), id(items))

2771200829288 2771200830920 2771200830008


In [11]:
d = dict(zip('abc', range(1,4)))

In [12]:
d

{'a': 1, 'b': 2, 'c': 3}

In [13]:
for k, v in d.items():
    print(k, v)
    del d[k]

a 1


RuntimeError: dictionary changed size during iteration

In [14]:
d

{'b': 2, 'c': 3}

In [15]:
d = dict(zip('abc', range(1, 4)))
for k , v in d.items():
    print(k, v)
    d['z'] = 100

a 1


RuntimeError: dictionary changed size during iteration

In [16]:
d

{'a': 1, 'b': 2, 'c': 3, 'z': 100}

In [17]:
d = dict(zip('abc', range(1, 4)))
for k , v in d.items():
    print(k, v)
    d[k] = 1000

a 1
b 2
c 3


In [18]:
d

{'a': 1000, 'b': 1000, 'c': 1000}

In [19]:
d = dict(zip('abc', range(1, 4)))
for k , v in d.items():
    print(k, v)
    d['c'] = 1000

a 1
b 2
c 1000


In [20]:
d

{'a': 1, 'b': 2, 'c': 1000}

In [21]:
d = dict(zip('abc', range(1, 4)))
for k in d.keys():
    print(d)
    del d[k]

{'a': 1, 'b': 2, 'c': 3}


RuntimeError: dictionary changed size during iteration

In [22]:
d

{'b': 2, 'c': 3}

In [23]:
d = dict(zip('abc', range(1, 4)))
for v in d.values():
    print(d)
    del d['c']

{'a': 1, 'b': 2, 'c': 3}


RuntimeError: dictionary changed size during iteration

In [24]:
d

{'a': 1, 'b': 2}

In [25]:
d = dict(zip('abc', range(1, 4)))
for key in d.keys():
    d[key] = 100

In [26]:
d

{'a': 100, 'b': 100, 'c': 100}

In [27]:
d = dict.fromkeys('python', 0)

In [28]:
d

{'p': 0, 'y': 0, 't': 0, 'h': 0, 'o': 0, 'n': 0}

In [29]:
for k in d:
    print(k)

p
y
t
h
o
n


In [30]:
d_iter = iter(d)
for k in d_iter:
    print(k)

p
y
t
h
o
n


In [31]:
d_iter = iter(d)
next(d_iter)

'p'

In [32]:
next(d_iter)

'y'

In [33]:
list(d_iter)

['t', 'h', 'o', 'n']

In [34]:
from timeit import timeit
from random import randint

In [38]:
d = {k: randint(0, 100) for k in range(10_000)}
keys = d.keys()

def iter_direct(d):
    for k in d:
        pass
    
def iter_view(d):
    for k in d.keys():
        pass
    
def iter_view_direct(view):
    for k in view:
        pass

print(timeit('iter_direct(d)', globals=globals(), number = 20_000))
print(timeit('iter_view(d)', globals=globals(), number = 20_000))
print(timeit('iter_view_direct(keys)', globals=globals(), number = 20_000))

1.5983371999991505
1.5928870999996434
1.5950013999990915


In [39]:
d = {'a': 1, 'b': 2, 'c': 3}
for k in d:
    print(k, d[k])

a 1
b 2
c 3


In [41]:
for k, v in d.items():
    print(k, v)

a 1
b 2
c 3


In [42]:
d = {k: randint(0, 100) for k in range(10_000)}

In [43]:
items = d.items()

def iterate_view(view):
    for k, v in view:
        pass
    
def iterate_clunky(d):
    for k in d:
        d[k]
        
print(timeit('iterate_view(items)', globals=globals(), number=5_000))
print(timeit('iterate_clunky(d)', globals=globals(), number=5_000))

0.6659134000001359
1.2961881999999605


In [44]:
d = {'a': 1, 'b': 2, 'c': 3}

In [45]:
for k, v in d.items():
    print(k, v**2)
    del d[k]

a 1


RuntimeError: dictionary changed size during iteration

In [46]:
d = {'a': 1, 'b': 2, 'c': 3}
keys = list(d.keys())
print(keys)

['a', 'b', 'c']


In [47]:
for k in keys:
    v = d[k]
    print(k, v**2)
    del d[k]

a 1
b 4
c 9


In [48]:
d

{}

In [50]:
d = {'a': 1, 'b': 2, 'c': 3}
for k in list(d.keys()):
    v = d.pop(k)
    print(k, v**2)

a 1
b 4
c 9


In [53]:
d = {'a': 1, 'b': 2, 'c': 3}
for _ in range(len(d)):
    key, value = d.popitem()
    print(key, value**2)

c 9
b 4
a 1


In [54]:
d

{}

In [55]:
d = {'a': 1, 'b': 2, 'c': 3}
while len(d) > 0:
    key, value = d.popitem()
    print(key, value**2)

c 9
b 4
a 1


In [56]:
d

{}

In [59]:
d = {'a': 1, 'b': 2, 'c': 3}
while True:
    try:
        key, value = d.popitem()
    except KeyError:
        break
    else:
        print(key, value**2)

c 9
b 4
a 1


In [60]:
d

{}

In [61]:
d = {'a':1, 'b': 2, 'c': 3}

In [62]:
for k, v in d.items():
    print(k, v)
    del d[k]
    d[k*2] = v ** 2

a 1
b 2
c 3


In [63]:
d

{'aa': 1, 'bb': 4, 'cc': 9}

This is really bad bad code, but it works. Fred does not reccomend!